In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from keras.callbacks import EarlyStopping
import tensorflow.keras.backend as kb
from libs.feature_extraction import *

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [3]:
from matplotlib import font_manager, rc

path = 'c:/Windows/Fonts/malgun.ttf'
font_name = font_manager.FontProperties(fname=path).get_name()
rc('font',family=font_name)

plt.style.use('fivethirtyeight')
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (20,10)

In [4]:
df = get_df('KOSPI', 'KOSPI-All', 'Add')
df

,close,대비,기관,외국인,개인,기타,금융투자,보험,투신,사모,...,low,volume,대비%,한국금리,천연가스,미국금리,다우존스,나스닥,구리,WTI
date,,,,,,,,,,,,,,,,,,,,,
2009-01-12,1156.75,-24.21,-105369,-100171,207988,-3455,-15603,-24208,-163712,-2151,...,1150.41,352670000,-2.05%,2.5,3.793,0.25,7223.98,1404.02,2.0370,67.16
2009-01-13,1167.71,10.96,-262362,30223,205073,26175,-2955,4193,-253006,-16493,...,1133.94,381510000,0.95%,2.5,3.550,0.25,7216.97,1462.11,2.0350,68.05
2009-01-14,1182.68,14.97,-5903,9837,-6527,2182,34413,-11807,-28832,-2658,...,1151.80,358880000,1.28%,2.5,3.695,0.25,7395.70,1491.22,2.0280,68.38
2009-01-15,1111.34,-71.34,-406404,-182570,563538,21116,-66342,31493,-439477,-5603,...,1108.15,366480000,-6.03%,2.5,3.604,0.25,7486.58,1483.48,2.0750,67.23
2009-01-16,1135.20,23.86,45743,-307461,236397,25424,5946,-7465,74338,8534,...,1109.69,335690000,2.15%,2.5,3.535,0.25,7400.80,1457.27,2.0700,63.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-24,2994.98,-75.11,-82457,631358,561382,-1115543,258837,-27013,-68615,-34963,...,2993.46,1560000000,-2.45%,0.5,2.777,0.25,31402.01,13119.43,4.2635,63.53
2021-02-25,3099.69,104.71,974913,974946,-1936090,-8948,1027147,-52027,5798,34583,...,3026.47,1280000000,3.50%,0.5,2.771,0.25,30932.37,13192.34,4.0925,61.50
2021-02-26,3012.95,-86.74,-1030068,-2829962,3778506,68921,-891584,26706,-38506,-21671,...,2988.28,1360000000,-2.80%,0.5,2.777,0.25,31535.51,13588.83,4.1130,60.64


In [5]:
df = Feature_Extraction(df)
df

Loading...
Finish Moving Average
Loading...
Finish Volatility
Loading...
Finish Volume
Loading...
Finish Momentum
Loading...
Finish All
Number of Features: 65
Number of Datas: 3001


,close,대비,기관,외국인,개인,기타,금융투자,보험,투신,사모,...,PVT,BollingerU,BollingerM,BollingerL,Typical,EMV,Mass,NVI,PVI,CV
date,,,,,,,,,,,,,,,,,,,,,
2009-01-12,1156.75,-24.21,-105369,-100171,207988,-3455,-15603,-24208,-163712,-2151,...,3.526700e+08,0.000000,0.000,0.000000,1161.450000,0.000000,0.000000,1000.00,1000.00,0.000000
2009-01-13,1167.71,10.96,-262362,30223,205073,26175,-2955,4193,-253006,-16493,...,3.562847e+08,0.000000,0.000,0.000000,1156.563333,-114.497916,0.000000,1000.00,1167.71,0.000000
2009-01-14,1182.68,14.97,-5903,9837,-6527,2182,34413,-11807,-28832,-2658,...,3.608856e+08,0.000000,0.000,0.000000,1173.000000,156.543246,0.000000,1182.68,1167.71,0.000000
2009-01-15,1111.34,-71.34,-406404,-182570,563538,21116,-66342,31493,-439477,-5603,...,3.387793e+08,0.000000,0.000,0.000000,1121.633333,-420.710216,0.000000,1182.68,1111.34,0.000000
2009-01-16,1135.20,23.86,45743,-307461,236397,25424,5946,-7465,74338,8534,...,3.459864e+08,1200.840819,1150.736,1100.631181,1126.893333,-31.411123,0.000000,1135.20,1111.34,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-24,2994.98,-75.11,-82457,631358,561382,-1115543,258837,-27013,-68615,-34963,...,9.537298e+08,3144.720606,3067.820,2990.919394,3026.830000,-139.795564,26.661155,2994.98,3070.09,-0.069874
2021-02-25,3099.69,104.71,974913,974946,-1936090,-8948,1027147,-52027,5798,34583,...,9.984810e+08,3150.520565,3070.426,2990.331435,3075.320000,116.755109,26.357024,3099.69,3070.09,-0.026551
2021-02-26,3012.95,-86.74,-1030068,-2829962,3778506,68921,-891584,26706,-38506,-21671,...,9.604235e+08,3132.221372,3051.492,2970.762628,3030.240000,-180.466360,26.067053,3099.69,3012.95,0.050881


In [6]:
df = df.drop(columns='대비%')
df['pred_price'] = np.where(df['close'].shift(-1) > df['close'], 1, 0)
df

,close,대비,기관,외국인,개인,기타,금융투자,보험,투신,사모,...,BollingerU,BollingerM,BollingerL,Typical,EMV,Mass,NVI,PVI,CV,pred_price
date,,,,,,,,,,,,,,,,,,,,,
2009-01-12,1156.75,-24.21,-105369,-100171,207988,-3455,-15603,-24208,-163712,-2151,...,0.000000,0.000,0.000000,1161.450000,0.000000,0.000000,1000.00,1000.00,0.000000,1
2009-01-13,1167.71,10.96,-262362,30223,205073,26175,-2955,4193,-253006,-16493,...,0.000000,0.000,0.000000,1156.563333,-114.497916,0.000000,1000.00,1167.71,0.000000,1
2009-01-14,1182.68,14.97,-5903,9837,-6527,2182,34413,-11807,-28832,-2658,...,0.000000,0.000,0.000000,1173.000000,156.543246,0.000000,1182.68,1167.71,0.000000,0
2009-01-15,1111.34,-71.34,-406404,-182570,563538,21116,-66342,31493,-439477,-5603,...,0.000000,0.000,0.000000,1121.633333,-420.710216,0.000000,1182.68,1111.34,0.000000,1
2009-01-16,1135.20,23.86,45743,-307461,236397,25424,5946,-7465,74338,8534,...,1200.840819,1150.736,1100.631181,1126.893333,-31.411123,0.000000,1135.20,1111.34,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-24,2994.98,-75.11,-82457,631358,561382,-1115543,258837,-27013,-68615,-34963,...,3144.720606,3067.820,2990.919394,3026.830000,-139.795564,26.661155,2994.98,3070.09,-0.069874,1
2021-02-25,3099.69,104.71,974913,974946,-1936090,-8948,1027147,-52027,5798,34583,...,3150.520565,3070.426,2990.331435,3075.320000,116.755109,26.357024,3099.69,3070.09,-0.026551,0
2021-02-26,3012.95,-86.74,-1030068,-2829962,3778506,68921,-891584,26706,-38506,-21671,...,3132.221372,3051.492,2970.762628,3030.240000,-180.466360,26.067053,3099.69,3012.95,0.050881,1


In [1]:
def init_population(n,c):
    return np.array([[math.ceil(e) for e in pop] for pop in (np.random.rand(n,c)-0.5)]), np.zeros((2,c))-1

def single_point_crossover(population):
    r,c, n = population.shape[0], population.shape[1], np.random.randint(1,population.shape[1])         
    for i in range(0,r,2):                
        population[i], population[i+1] = np.append(population[i][0:n],population[i+1][n:c]),np.append(population[i+1][0:n],population[i][n:c])        
    return population

def flip_mutation(population):
    return population.max() - population

def random_selection(population):
    r = population.shape[0]
    new_population = population.copy()    
    for i in range(r):        
        new_population[i] = population[np.random.randint(0,r)]
    return new_population

In [9]:
def get_fitness(data, feature_list, target, population):    
    fitness = []
    for i in range(population.shape[0]):        
        columns = [feature_list[j] for j in range(population.shape[1]) if population[i,j]==1]                    
        fitness.append(predictive_model(data[columns], data[target]))                
    return fitness

def predictive_model(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=7)
    lr = LogisticRegression(solver='liblinear', max_iter=100, random_state=7)
    lr.fit(X_train,y_train)    
    return accuracy_score(y_test, lr.predict(X_test))

In [15]:
def memorize(pop, memory):      
    return np.append(memory, pop.reshape(1,memory.shape[1]), axis=0)

def replace_duplicate(population, memory):    
    for i in range(population.shape[0]):         
        counter = 0                
        while population.shape[1] in sum((memory==population[i]).astype(int).T) and counter<100:                                
            population[i] = np.array([math.ceil(k) for k in (np.random.rand(population.shape[1])-0.5)])                    
            counter += 1                    
        memory = memorize(population[i], memory)        
    return population, memory

In [10]:
def ga(data, feature_list, target, n, max_iter):
    
    c = len(feature_list)
    
    population, memory = init_population(n,c)
    population, memory = replace_duplicate(population, memory)    
    
    fitness    = get_fitness(data, feature_list, target, population)    
    
    optimal_value    = max(fitness)
    optimal_solution = population[np.where(fitness==optimal_value)][0]    
    
    for i in range(max_iter):                
        population = random_selection(population)
        population = single_point_crossover(population)                        
        if np.random.rand() < 0.3:
            population = flip_mutation(population)   
        
        population, memory = replace_duplicate(population, memory)
                
        fitness = get_fitness(data, feature_list, target, population)
                
        if max(fitness) > optimal_value:
            optimal_value    = max(fitness)
            optimal_solution = population[np.where(fitness==optimal_value)][0]                               
        
    return optimal_solution, optimal_value

In [11]:
feature_list = df.columns.difference(["close"])
target = 'pred_price'

In [20]:
feature_set, acc_score = ga(df, feature_list, target, 10, 1000)

# Filter Selected Features
feature_set = [feature_list[i] for i in range(len(feature_list)) if feature_set[i]==1]

# Print List of Features
print('Optimal Feature Set\n',feature_set,'\nOptimal Accuracy =', round(acc_score*100), '%')

Optimal Feature Set
 ['ATR', 'EMA_10', 'MFI', 'Mass', 'NVI', 'PVI', 'ROC', 'SMA_20', 'SMA_200', 'SOD', 'ULTOSC', 'WillA/D', 'WillR', 'high', 'pred_price', '구리', '금융투자', '나스닥', '다우존스', '대비', '보험', '연기금', '외국인'] 
Optimal Accuracy = 100 %


In [19]:
import math
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [21]:
feature_list = df.columns.difference(["close", 'pred_price'])
target = 'pred_price'

In [22]:
feature_set, acc_score = ga(df, feature_list, target, 10, 1000)

# Filter Selected Features
feature_set = [feature_list[i] for i in range(len(feature_list)) if feature_set[i]==1]

# Print List of Features
print('Optimal Feature Set\n',feature_set,'\nOptimal Accuracy =', round(acc_score*100), '%')

Optimal Feature Set
 ['ADX', 'BollingerM', 'BollingerU', 'CCI', 'CV', 'EMA_100', 'EMV', 'MACDH', 'MACDS', 'MOM', 'NVI', 'ROC', 'RSI', 'SMA_100', 'SMA_50', 'SOJ', 'TSI', 'Typical', 'high', 'low', 'open', '구리', '기타', '나스닥', '미국금리', '천연가스', '투신'] 
Optimal Accuracy = 59 %


In [23]:
feature_list = df.columns.difference(['pred_price'])
target = 'pred_price'

In [24]:
feature_set, acc_score = ga(df, feature_list, target, 10, 1000)

# Filter Selected Features
feature_set = [feature_list[i] for i in range(len(feature_list)) if feature_set[i]==1]

# Print List of Features
print('Optimal Feature Set\n',feature_set,'\nOptimal Accuracy =', round(acc_score*100), '%')

Optimal Feature Set
 ['ADX', 'BollingerL', 'BollingerM', 'CCI', 'EMV', 'MOM', 'Mass', 'NVI', 'PVI', 'ROC', 'SMA_20', 'SMA_50', 'SOJ', 'TSI', 'Typical', 'WTI', 'WillR', 'high', 'low', '기타', '나스닥', '미국금리', '사모', '연기금', '외국인', '은행', '투신'] 
Optimal Accuracy = 58 %


In [25]:
!pip install feature-selection-ga

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=cec1babeaf8eaee7afd645019dd9c4dc1188aef14ed95ba9d3b9e4077ec28da7
  Stored in directory: c:\users\vfgtr\appdata\local\pip\cache\wheels\46\ef\c3\157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [26]:
from sklearn import linear_model
from feature_selection_ga import FeatureSelectionGA, FitnessFunction

In [7]:
feature_list = df.columns.difference(["close", 'pred_price'])
target = 'pred_price'
X = df[feature_list]
y = df[target]
X.shape, y.shape

((3001, 63), (3001,))

In [30]:
model = linear_model.LogisticRegression(solver='lbfgs', multi_class='auto')
fsga = FeatureSelectionGA(model,X,y, ff_obj = FitnessFunction())
pop = fsga.generate(100)

EVOLVING.......


TypeError: '(slice(None, None, None), array([ 0,  3,  4,  8, 10, 12, 13, 14, 16, 17, 18, 21, 22, 24, 25, 29, 30,
       31, 34, 35, 36, 38, 39, 41, 46, 51, 53, 54, 56, 59, 60],
      dtype=int64))' is an invalid key

In [31]:
import numpy
import sklearn.svm

def reduce_features(solution, features):
    selected_elements_indices = numpy.where(solution == 1)[0]
    reduced_features = features[:, selected_elements_indices]
    return reduced_features


def classification_accuracy(labels, predictions):
    correct = numpy.where(labels == predictions)[0]
    accuracy = correct.shape[0]/labels.shape[0]
    return accuracy


def cal_pop_fitness(pop, features, labels, train_indices, test_indices):
    accuracies = numpy.zeros(pop.shape[0])
    idx = 0

    for curr_solution in pop:
        reduced_features = reduce_features(curr_solution, features)
        train_data = reduced_features[train_indices, :]
        test_data = reduced_features[test_indices, :]

        train_labels = labels[train_indices]
        test_labels = labels[test_indices]

        SV_classifier = sklearn.svm.SVC(gamma='scale')
        SV_classifier.fit(X=train_data, y=train_labels)

        predictions = SV_classifier.predict(test_data)
        accuracies[idx] = classification_accuracy(test_labels, predictions)
        idx = idx + 1
    return accuracies

def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = numpy.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
    return parents


def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually, it is at the center.
    crossover_point = numpy.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring


def mutation(offspring_crossover, num_mutations=2):
    mutation_idx = numpy.random.randint(low=0, high=offspring_crossover.shape[1], size=num_mutations)
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        offspring_crossover[idx, mutation_idx] = 1 - offspring_crossover[idx, mutation_idx]
    return offspring_crossover

In [33]:
num_samples = X.shape[0]
num_feature_elements = X.shape[1]
train_indices = np.arange(1, num_samples, 4)
test_indices = np.arange(0, num_samples, 4)
train_indices.shape[0], test_indices.shape[0]

(750, 751)

In [36]:
sol_per_pop = 8 # Population size.
num_parents_mating = 4 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
# Defining the population shape.
pop_shape = (sol_per_pop, num_feature_elements)
 
# Creating the initial population.
new_population = numpy.random.randint(low=0, high=2, size=pop_shape)
print(new_population.shape)
 
best_outputs = []
num_generations = 100

(8, 63)


In [38]:
for generation in range(num_generations):
    print("Generation : ", generation)
    # Measuring the fitness of each chromosome in the population.
    fitness = cal_pop_fitness(new_population, X, y, train_indices, test_indices)

    best_outputs.append(numpy.max(fitness))
    # The best result in the current iteration.
    print("Best result : ", best_outputs[-1])

    # Selecting the best parents in the population for mating.
    parents = select_mating_pool(new_population, fitness, num_parents_mating)

    # Generating next generation using crossover.
    offspring_crossover = crossover(parents, offspring_size=(pop_shape[0]-parents.shape[0], num_feature_elements))

    # Adding some variations to the offspring using mutation.
    offspring_mutation = mutation(offspring_crossover, num_mutations=num_mutations)

    # Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation

Generation :  0


TypeError: '(slice(None, None, None), array([ 0,  2,  3,  4,  7,  8,  9, 12, 13, 14, 15, 17, 18, 21, 25, 26, 27,
       28, 29, 31, 32, 33, 42, 46, 48, 49, 50, 52, 53, 54, 55, 56, 58, 59,
       60, 61, 62], dtype=int64))' is an invalid key

In [8]:
X.shape, y.shape

((3001, 63), (3001,))

In [9]:
from sklearn.metrics import *
from sklearn.model_selection import *
from genetic_selection import GeneticSelectionCV

In [39]:
from sklearn.metrics import *
mcc = make_scorer(matthews_corrcoef)
estimator = LogisticRegression(solver = "liblinear", C = 6, tol = 1, fit_intercept = True)

In [40]:
from sklearn.model_selection import *
report = pd.DataFrame()
nofeats = [] 
chosen_feats = [] 
cvscore = [] 
rkf = RepeatedStratifiedKFold(n_repeats = 2, n_splits = 10)

In [ ]:
X.shape, y.shape

In [44]:
!pip install sklearn-genetic

In [45]:
from genetic_selection import GeneticSelectionCV

In [46]:
for i in range(2,63):
  
  selector = GeneticSelectionCV(estimator,
                                cv = rkf,
                                verbose = 0,
                                scoring = mcc,
                                max_features = i,
                                n_population = 200,
                                crossover_proba = 0.5,
                                mutation_proba = 0.2,
                                n_generations = 10,
                                crossover_independent_proba=0.5,
                                mutation_independent_proba=0.05,
                                #tournament_size = 3,
                                n_gen_no_change=10,
                                caching=True,
                                n_jobs=-1)
  selector = selector.fit(X, y)
  genfeats = X.columns[selector.support_]
  genfeats = list(genfeats)
  print("Chosen Feats:  ", genfeats)

  cv_score = selector.generation_scores_[-1]
  nofeats.append(len(genfeats)) 
  chosen_feats.append(genfeats) 
  cvscore.append(cv_score)

Chosen Feats:   ['ULTOSC', '금융투자']
Chosen Feats:   ['MOM', 'SMA_100']
Chosen Feats:   ['CV', '구리', '한국금리']
Chosen Feats:   ['TRIX', '구리', '한국금리']
Chosen Feats:   ['CCI', 'EMA_10', 'WTI', '개인', '기타', '다우존스']
Chosen Feats:   ['BollingerL', 'EMA_200', '기타', '사모']
Chosen Feats:   ['NVI', '사모']
Chosen Feats:   ['EMA_10', 'SMA_10', 'SMA_20', 'TSI', 'ULTOSC', 'WillA/D', '사모']
Chosen Feats:   ['ATR', 'EMA_200', 'MACD', 'MFI', 'SOJ', 'WTI', 'low', '미국금리', '사모']
Chosen Feats:   ['금융투자']
Chosen Feats:   ['CCI', 'EMA_200', 'MACDS', 'Mass', 'ROC', 'SMA_10', 'SMA_20', 'SOJ', 'TRIX', 'WillR', '은행', '한국금리']
Chosen Feats:   ['EMA_20', 'MACD', 'RSI', 'SMA_20', 'SOK', '기타', '나스닥', '사모', '은행']
Chosen Feats:   ['한국금리']
Chosen Feats:   ['CCI', 'EMA_10', 'EMA_200', 'PVI', 'TSI', 'WillR', 'open', '사모', '연기금', '천연가스']
Chosen Feats:   ['ADX', 'CCI', 'CV', 'EMA_200', 'EMV', 'NVI', 'PVI', 'ROC', 'SOJ', 'SOK', 'TRIX', 'WillR', 'high', '대비', '사모', '연기금']
Chosen Feats:   ['BollingerL', 'MACD', 'ROC', 'SOD', '사모', '한

In [47]:
report["No of Feats"] = nofeats
report["Chosen Feats"] = chosen_feats
report["Scores"] = cvscore

In [64]:
report.to_csv('report.csv', encoding='utf-8-sig')

In [48]:
report

,No of Feats,Chosen Feats,Scores
0,2,"[ULTOSC, 금융투자]",0.030206
1,2,"[MOM, SMA_100]",0.028635
2,3,"[CV, 구리, 한국금리]",0.041947
3,3,"[TRIX, 구리, 한국금리]",0.038411
4,6,"[CCI, EMA_10, WTI, 개인, 기타, 다우존스]",0.031949
...,...,...,...
56,10,"[ATR, CCI, CV, EMA_50, MFI, MOM, TRIX, low, 기타...",0.033100
57,8,"[ATR, EMA_100, EMA_50, MACDS, Typical, WillR, ...",0.036824
58,11,"[BollingerM, EMA_10, Mass, NVI, PVI, SMA_100, ...",0.039241
59,13,"[ATR, BollingerM, MACD, Mass, NVI, SMA_20, SMA...",0.035186


In [10]:
report = pd.read_csv('report.csv')
report

,Unnamed: 0,No of Feats,Chosen Feats,Scores
0,0,2,"['ULTOSC', '금융투자']",0.030206
1,1,2,"['MOM', 'SMA_100']",0.028635
2,2,3,"['CV', '구리', '한국금리']",0.041947
3,3,3,"['TRIX', '구리', '한국금리']",0.038411
4,4,6,"['CCI', 'EMA_10', 'WTI', '개인', '기타', '다우존스']",0.031949
...,...,...,...,...
56,56,10,"['ATR', 'CCI', 'CV', 'EMA_50', 'MFI', 'MOM', '...",0.033100
57,57,8,"['ATR', 'EMA_100', 'EMA_50', 'MACDS', 'Typical...",0.036824
58,58,11,"['BollingerM', 'EMA_10', 'Mass', 'NVI', 'PVI',...",0.039241
59,59,13,"['ATR', 'BollingerM', 'MACD', 'Mass', 'NVI', '...",0.035186


In [11]:
import pycaret
pycaret.__version__

'2.3.1'

In [12]:
report = report.drop('Unnamed: 0', axis=1)

In [34]:
from pycaret.datasets import get_data
from pycaret.classification import *
report["Scores"] = np.round(report["Scores"], 3)
report.sort_values(by = "Scores", ascending = False, inplace = True)
#report.index
ga_feats = report.iloc[0]["Chosen Feats"]

In [44]:
ga_featss = ga_feats[:-2] + "', 'pred_price'" + "]"
ga_featss

"['EMA_100', 'EMA_200', 'Mass', 'PVI', 'SMA_10', 'SMA_50', 'SOK', 'Typical', '사모', '천연가스', 'pred_price']"

In [42]:
ga_feats[:-2]

"['EMA_100', 'EMA_200', 'Mass', 'PVI', 'SMA_10', 'SMA_50', 'SOK', 'Typical', '사모', '천연가스"

In [36]:
feature_list = df.columns.difference(["close"])
target = 'pred_price'
X = df[feature_list]
y = df[target]
X.shape, y.shape

((3001, 64), (3001,))

In [46]:
feature_list = ['EMA_100', 'EMA_200', 'Mass', 'PVI', 'SMA_10', 'SMA_50', 'SOK', 'Typical', '사모', '천연가스', 'pred_price']

In [47]:
ename = setup(data = df[feature_list], target = "pred_price", 
              test_data = None,
              fold_strategy = "stratifiedkfold",
              fold_shuffle = True, 
              use_gpu = True,
              normalize = True,
              categorical_features = None,
              
              #pca = True,
              #pca_method = "kernel",
              #pca_components = 5, 
              preprocess = False,
              html = True,
              #POLYNOMIAL
              #polynomial_features = True,
              #polynomial_degree = 2, 
              #transformation
              
              #transformation = True,
              #feature_selection = True, 
              feature_interaction = True,
          
              fix_imbalance = True,
              #fix_imbalance_method  = imblearn.over_sampling.SMOTE(),
              imputation_type='simple',
              verbose = True,
              )

,Description,Value
0,session_id,5769
1,Target,pred_price
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(3001, 11)"
5,Missing Values,False
6,Numeric Features,10
7,Categorical Features,0
8,Transformed Train Set,"(2100, 10)"
9,Transformed Test Set,"(901, 10)"


In [48]:
rskf = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 20)
best_model = compare_models(sort = "MCC", round = 2, 
                            fold = rskf,
                            #include = ["rf","catboost"],
                            #n_select = 1
                            )

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.52,0.52,0.73,0.53,0.61,0.03,0.03,0.10
ridge,Ridge Classifier,0.52,0.00,0.74,0.53,0.62,0.02,0.03,0.01
lda,Linear Discriminant Analysis,0.52,0.52,0.74,0.53,0.62,0.02,0.03,0.01
qda,Quadratic Discriminant Analysis,0.50,0.52,0.35,0.54,0.39,0.02,0.02,0.01
knn,K Neighbors Classifier,0.51,0.51,0.55,0.53,0.54,0.01,0.01,0.10
nb,Naive Bayes,0.50,0.51,0.35,0.53,0.40,0.01,0.01,0.01
rf,Random Forest Classifier,0.51,0.50,0.54,0.53,0.53,0.01,0.01,1.11
ada,Ada Boost Classifier,0.51,0.50,0.60,0.53,0.56,0.01,0.01,0.17
gbc,Gradient Boosting Classifier,0.51,0.51,0.60,0.53,0.56,0.01,0.01,0.43
dt,Decision Tree Classifier,0.50,0.50,0.52,0.52,0.52,0.00,0.00,0.03


In [49]:
feature_list = df.columns.difference(["close"])
target = 'pred_price'
X = df[feature_list]
y = df[target]
X.shape, y.shape

((3001, 64), (3001,))

In [50]:
ename = setup(data = df[feature_list], target = "pred_price", 
              test_data = None,
              fold_strategy = "stratifiedkfold",
              fold_shuffle = True, 
              use_gpu = True,
              normalize = True,
              categorical_features = None,
              
              #pca = True,
              #pca_method = "kernel",
              #pca_components = 5, 
              preprocess = False,
              html = True,
              #POLYNOMIAL
              #polynomial_features = True,
              #polynomial_degree = 2, 
              #transformation
              
              #transformation = True,
              #feature_selection = True, 
              feature_interaction = True,
          
              fix_imbalance = True,
              #fix_imbalance_method  = imblearn.over_sampling.SMOTE(),
              imputation_type='simple',
              verbose = True,
              )

,Description,Value
0,session_id,5474
1,Target,pred_price
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(3001, 64)"
5,Missing Values,False
6,Numeric Features,63
7,Categorical Features,0
8,Transformed Train Set,"(2100, 63)"
9,Transformed Test Set,"(901, 63)"


In [51]:
rskf = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 20)
best_model = compare_models(sort = "MCC", round = 2, 
                            fold = rskf,
                            #include = ["rf","catboost"],
                            #n_select = 1
                            )

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.55,0.57,0.66,0.57,0.61,0.09,0.09,0.04
et,Extra Trees Classifier,0.52,0.52,0.58,0.55,0.56,0.04,0.04,1.18
ridge,Ridge Classifier,0.53,0.00,0.69,0.55,0.61,0.03,0.03,0.02
rf,Random Forest Classifier,0.52,0.52,0.60,0.55,0.57,0.03,0.03,1.16
qda,Quadratic Discriminant Analysis,0.51,0.51,0.52,0.55,0.51,0.01,0.02,0.05
knn,K Neighbors Classifier,0.51,0.51,0.56,0.54,0.55,0.01,0.01,0.14
dt,Decision Tree Classifier,0.51,0.50,0.53,0.54,0.54,0.01,0.01,0.10
ada,Ada Boost Classifier,0.51,0.51,0.60,0.54,0.57,0.01,0.01,0.55
svm,SVM - Linear Kernel,0.50,0.00,0.49,0.48,0.35,-0.00,0.00,0.03
gbc,Gradient Boosting Classifier,0.51,0.51,0.64,0.54,0.58,0.00,0.00,2.27


In [25]:
from pycaret.regression import *

In [27]:
feature_list = df.columns.difference(['pred_price'])
target = 'close'
X = df[feature_list]
y = df[target]
X.shape, y.shape

((3001, 64), (3001,))

In [32]:
reg1 = setup(data = df[feature_list], target = "close", 
              test_data = None,
              use_gpu = True,
              normalize = True,
              
              #pca = True,
              #pca_method = "kernel",
              #pca_components = 5, 
              preprocess = False,
              html = True,
              #POLYNOMIAL
              #polynomial_features = True,
              #polynomial_degree = 2, 
              #transformation
              
              #transformation = True,
              feature_selection = True, 
              feature_interaction = True,
          
              #fix_imbalance_method  = imblearn.over_sampling.SMOTE(),
              imputation_type='simple',
              verbose = True,
              )

,Description,Value
0,session_id,7008
1,Target,close
2,Original Data,"(3001, 64)"
3,Missing Values,False
4,Numeric Features,63
5,Categorical Features,0
6,Transformed Train Set,"(2100, 63)"
7,Transformed Test Set,"(901, 63)"
8,Shuffle Train-Test,True
9,Stratify Train-Test,False


In [33]:
best_model = compare_models(fold=5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
omp,Orthogonal Matching Pursuit,2.6383,13.3218,3.6387,0.9998,0.0019,0.0013,0.0100
lasso,Lasso Regression,3.4390,27.5245,5.2036,0.9997,0.0028,0.0017,0.0320
en,Elastic Net,3.4527,27.7017,5.2211,0.9997,0.0029,0.0018,0.0340
et,Extra Trees Regressor,4.4297,48.0797,6.8811,0.9995,0.0036,0.0023,1.1020
rf,Random Forest Regressor,4.6946,53.4913,7.2489,0.9994,0.0038,0.0024,1.5000
gbr,Gradient Boosting Regressor,4.6613,49.0641,6.9734,0.9994,0.0036,0.0024,2.1880
dt,Decision Tree Regressor,6.5343,110.5000,10.4772,0.9987,0.0052,0.0033,0.0760
br,Bayesian Ridge,9.6401,373.2889,13.2089,0.9960,0.0066,0.0048,0.0860
ada,AdaBoost Regressor,21.4233,741.5735,26.9216,0.9915,0.0145,0.0110,0.6900
llar,Lasso Least Angle Regression,28.0243,1732.9129,41.4484,0.9808,0.0234,0.0148,0.0080


In [31]:
lightgbm = create_model('lightgbm')

IntProgress(value=0, description='Processing: ', max=4)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,22:44:08
Status,. . . . . . . . . . . . . . . . . .,Compiling Final Models
Estimator,. . . . . . . . . . . . . . . . . .,Linear Discriminant Analysis


,MAE,MSE,RMSE,R2,RMSLE,MAPE


LightGBMError: Do not support non-ASCII characters in feature name.